<a href="https://colab.research.google.com/github/iim0663418/LLMDataOptimization/blob/main/Pandoc_%E5%AF%A6%E7%8F%BE%E6%A0%BC%E5%BC%8F%E4%BF%9D%E7%95%99%E7%9A%84%E6%96%87%E6%AA%94%E8%BD%89%E6%8F%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pandoc 是一個通用的文檔轉換工具，可以在多種格式之間轉換文檔，如 DOCX、ODT、Markdown、HTML、LaTeX 等。它能夠保留文檔的格式，並提供多種輸出選項。
這段程式碼只支援 DOCX、ODT 運用：
1. 先將文檔轉換為 HTML 格式，保留格式信息。
2. 然後使用自定義的解析器從 HTML 中提取所需的結構化數據，並進一步轉換為其他格式。</br>

方法論轉換成純文字檔案的過程。

同時加入將 json 轉換成語言模型友善的 .txt 方法，藉此保留原有的格式

In [3]:
!sudo apt-get install pandoc
!pip install beautifulsoup4


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc-data
Suggested packages:
  texlive-latex-recommended texlive-xetex texlive-luatex pandoc-citeproc
  texlive-latex-extra context wkhtmltopdf librsvg2-bin groff ghc nodejs php
  python ruby libjs-mathjax libjs-katex citation-style-language-styles
The following NEW packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc
  pandoc-data
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 20.6 MB of archives.
After this operation, 156 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [115 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm-extensions0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [25.1 kB

In [7]:
import os

def create_directories():
    input_dir = "/content/input"
    output_dir = "/content/output"
    txtOutput_dir = "/content/txtOutput"

    # 創建 /content/input 目錄
    os.makedirs(input_dir, exist_ok=True)
    print(f"Directory created: {input_dir}")

    # 創建 /content/output 目錄
    os.makedirs(output_dir, exist_ok=True)
    print(f"Directory created: {output_dir}")
    # 創建 /content/output 目錄
    os.makedirs(txtOutput_dir, exist_ok=True)
    print(f"Directory created: {txtOutput_dir}")

# 執行函數創建目錄
create_directories()


Directory created: /content/input
Directory created: /content/output
Directory created: /content/txtOutput


In [5]:
import subprocess
import os

def convert_to_html(input_file, output_file):
    # 檢查文件類型
    file_extension = os.path.splitext(input_file)[1].lower()

    if file_extension not in ['.docx', '.odt']:
        raise ValueError(f"Unsupported file format: {file_extension}")

    # 使用 Pandoc 將文檔轉換為 HTML
    subprocess.run(['pandoc', '-o', output_file, input_file])
    print(f"Converted {input_file} to {output_file}")

# 使用範例
# input_file = "example.docx"
# output_file = "example.html"
# convert_to_html(input_file, output_file)

from bs4 import BeautifulSoup

def parse_html_and_extract_data(html_file):
    # 打開並解析 HTML 文件
    with open(html_file, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # 示例：提取所有的標題和段落
    data = {}
    data['titles'] = [title.get_text() for title in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
    data['paragraphs'] = [para.get_text() for para in soup.find_all('p')]

    # 提取列表
    data['lists'] = []
    for ul in soup.find_all('ul'):
        items = [li.get_text() for li in ul.find_all('li')]
        data['lists'].append(items)

    # 提取表格（如果需要）
    data['tables'] = []
    for table in soup.find_all('table'):
        table_data = []
        for row in table.find_all('tr'):
            columns = [col.get_text() for col in row.find_all(['td', 'th'])]
            table_data.append(columns)
        data['tables'].append(table_data)

    return data

# 使用範例
# html_file = "example.html"
# extracted_data = parse_html_and_extract_data(html_file)
# print(extracted_data)

#轉換為 JSON 格式：
import json

def save_data_as_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Data saved as JSON to {output_file}")

# 使用範例
# json_output_file = "extracted_data.json"
# save_data_as_json(extracted_data, json_output_file)

#轉換為 CSV 格式（僅表格數據示例）：
import csv

def save_table_data_as_csv(data, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for table in data['tables']:
            writer.writerows(table)
            writer.writerow([])  # 每個表格之間空一行
    print(f"Table data saved as CSV to {output_file}")

# 使用範例
# csv_output_file = "extracted_data.csv"
# save_table_data_as_csv(extracted_data, csv_output_file)


In [6]:
import os
import json
import traceback

def convert_and_extract(input_file, html_output_file, json_output_file, csv_output_file):
    # 第一步：將文檔轉換為 HTML 格式
    convert_to_html(input_file, html_output_file)

    # 第二步：解析 HTML 並提取結構化數據
    extracted_data = parse_html_and_extract_data(html_output_file)

    # 第三步：將數據保存為 JSON 格式
    save_data_as_json(extracted_data, json_output_file)

    # 第四步：將表格數據保存為 CSV 格式
    save_table_data_as_csv(extracted_data, csv_output_file)

def convert_and_extract_with_error_handling(input_file, html_output_file, json_output_file, csv_output_file):
    try:
        # 執行轉換和提取
        convert_and_extract(input_file, html_output_file, json_output_file, csv_output_file)
        print(f"Successfully processed {input_file}")
    except Exception as e:
        # 捕獲所有異常，記錄錯誤並繼續
        error_message = f"Failed to process {input_file}: {str(e)}"
        print(error_message)
        with open("error_log.txt", "a", encoding="utf-8") as log_file:
            log_file.write(f"{error_message}\n")
            log_file.write(traceback.format_exc())  # 記錄完整的堆棧跟踪
            log_file.write("\n")

def batch_convert_and_extract_with_error_handling(input_folder, output_folder, summary_file):
    summary_data = []

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.docx', '.odt')):  # 支援 DOCX 和 ODT 文件格式
            input_file = os.path.join(input_folder, filename)
            base_name = os.path.splitext(filename)[0]

            html_output_file = os.path.join(output_folder, f"{base_name}.html")
            json_output_file = os.path.join(output_folder, f"{base_name}.json")
            csv_output_file = os.path.join(output_folder, f"{base_name}.csv")

            # 對每個文件進行轉換和提取，並添加錯誤處理
            convert_and_extract_with_error_handling(input_file, html_output_file, json_output_file, csv_output_file)

            # 如果 JSON 文件成功生成，則將其匯總
            if os.path.exists(json_output_file):
                with open(json_output_file, 'r', encoding='utf-8') as f:
                    file_data = json.load(f)
                    summary_data.append({base_name: file_data})

    # 將匯總結果保存為 JSON 文件
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary_data, f, ensure_ascii=False, indent=4)
    print(f"Summary saved to {summary_file}")

# 使用範例
input_folder = "/content/input"
output_folder = "/content/output"
summary_file = "/content/output/summary.json"

batch_convert_and_extract_with_error_handling(input_folder, output_folder, summary_file)


Converted /content/input/附件1-113年第1次資料開放工作小組會議紀錄 (1).odt to /content/output/附件1-113年第1次資料開放工作小組會議紀錄 (1).html
Data saved as JSON to /content/output/附件1-113年第1次資料開放工作小組會議紀錄 (1).json
Table data saved as CSV to /content/output/附件1-113年第1次資料開放工作小組會議紀錄 (1).csv
Successfully processed /content/input/附件1-113年第1次資料開放工作小組會議紀錄 (1).odt
Converted /content/input/113年第2次資料開放工作小組會議紀錄.odt to /content/output/113年第2次資料開放工作小組會議紀錄.html
Data saved as JSON to /content/output/113年第2次資料開放工作小組會議紀錄.json
Table data saved as CSV to /content/output/113年第2次資料開放工作小組會議紀錄.csv
Successfully processed /content/input/113年第2次資料開放工作小組會議紀錄.odt
Converted /content/input/附件1-1111006業務會議紀錄.odt to /content/output/附件1-1111006業務會議紀錄.html
Data saved as JSON to /content/output/附件1-1111006業務會議紀錄.json
Table data saved as CSV to /content/output/附件1-1111006業務會議紀錄.csv
Successfully processed /content/input/附件1-1111006業務會議紀錄.odt
Summary saved to /content/output/summary.json


In [8]:
import json
import os

def json_to_txt(json_file, output_file):
    try:
        # 讀取 JSON 文件
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 開始寫入 TXT 文件
        with open(output_file, 'w', encoding='utf-8') as f:
            # 寫入標題
            if 'titles' in data:
                for title in data['titles']:
                    f.write(f"# {title}\n\n")  # 使用 # 表示標題

            # 寫入段落
            if 'paragraphs' in data:
                for paragraph in data['paragraphs']:
                    f.write(f"{paragraph}\n\n")  # 每段後加兩個換行

            # 寫入列表
            if 'lists' in data:
                for lst in data['lists']:
                    for item in lst:
                        f.write(f"- {item}\n")  # 使用 - 表示列表項
                    f.write("\n")  # 列表之間加一個換行

            # 寫入表格
            if 'tables' in data:
                for table in data['tables']:
                    for row in table:
                        f.write(" | ".join(row) + "\n")  # 使用 | 分隔表格列
                    f.write("\n")  # 表格之間加一個換行

        print(f"Converted {json_file} to {output_file}")

    except Exception as e:
        print(f"Failed to process {json_file}: {e}")

# 使用範例
# json_file = "extracted_data.json"
# output_file = "output_data.txt"
# json_to_txt(json_file, output_file)

def batch_json_to_txt(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            json_file = os.path.join(input_folder, filename)
            output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
            json_to_txt(json_file, output_file)

# 使用範例
input_folder = "/content/output"
output_folder = "/content/txtOutput"

batch_json_to_txt(input_folder, output_folder)


Converted /content/output/附件1-1111006業務會議紀錄.json to /content/txtOutput/附件1-1111006業務會議紀錄.txt
Converted /content/output/附件1-113年第1次資料開放工作小組會議紀錄 (1).json to /content/txtOutput/附件1-113年第1次資料開放工作小組會議紀錄 (1).txt
Converted /content/output/113年第2次資料開放工作小組會議紀錄.json to /content/txtOutput/113年第2次資料開放工作小組會議紀錄.txt
Converted /content/output/summary.json to /content/txtOutput/summary.txt


In [ ]:
import shutil
import os

def zip_output_directory():
    output_dir = "/content/output"
    zip_file_path = "/content/output_archive.zip"

    # 確保 output 目錄存在
    if os.path.exists(output_dir):
        # 使用 shutil.make_archive 創建 ZIP 文件
        shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', output_dir)
        print(f"Directory {output_dir} has been zipped into {zip_file_path}")
    else:
        print(f"Directory {output_dir} does not exist")

# 執行打包
zip_output_directory()
